In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Add, GlobalAveragePooling2D, Dense, Flatten, Conv2D, Lambda, Input, BatchNormalization, Activation
from tensorflow.keras.optimizers import schedules, SGD
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
# def load_dataset():
    """
    Load the FER2013 dataset.
    """
file_path = '/kaggle/input/fer2013/fer2013.csv'
data = pd.read_csv(file_path)

pixels = data['pixels'].tolist()
width, height = 48, 48
faces = []
for pixel_sequence in pixels:
    face = np.asarray([int(pixel) for pixel in pixel_sequence.split(' ')]).reshape(width, height)
    face = np.stack((face,)*3, axis=-1)  # Convert to RGB
    faces.append(face)
faces = np.array(faces)
faces = faces.astype('float32') / 255.0

emotions = pd.get_dummies(data['emotion']).values

# Split the data into training and testing sets
train_size = int(0.8 * len(faces))
input_train, input_test = faces[:train_size], faces[train_size:]
target_train, target_test = emotions[:train_size], emotions[train_size:]

#     return (X_train, y_train), (X_test, y_test)

# (input_train, target_train), (input_test, target_test) = (X_train, y_train), (X_test, y_test)

input_train = X_train
target_train = y_train
input_test = X_test
target_test = y_test
# print("",input_train.shape)

width, height, dim = 48, 48, 3
num_classes = 7
# def preprocess_image(image):
#     image = tf.image.resize(image, (48, 48))
#     return image

# Data augmentation: perform zero padding on datasets
# paddings = tf.constant([[0, 0], [4, 4], [4, 4], [0, 0]])
# input_train = tf.pad(input_train, paddings, mode="CONSTANT")

# Data generator for training data with additional augmentations
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=10,          # Random rotations
    width_shift_range=0.1,      # Random horizontal shifts
    height_shift_range=0.1,     # Random vertical shifts
    zoom_range=0.1,             # Random zoom
    shear_range=0.1,            # Random shear
    rescale=1./255
#     preprocessing_function=preprocess_image
)

# Generate training and validation batches
train_batches = train_generator.flow(input_train, target_train, batch_size=32, subset="training")
validation_batches = train_generator.flow(input_train, target_train, batch_size=32, subset="validation")
# batch_x, batch_y = next(train_batches)
# print(batch_x.shape)
# Data generator for testing data
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

# Generate test batches
test_batches = test_generator.flow(input_test, target_test, batch_size=32)

In [ ]:
def random_crop(img, random_crop_size):
    """
    Perform a random crop on the image.
    """
    height, width = img.shape[:2]
    dy, dx = random_crop_size
    x = np.random.randint(0, width - dx + 1)
    y = np.random.randint(0, height - dy + 1)
    return img[y:(y+dy), x:(x+dx), :]


def crop_generator(batches, crop_length):
    """
    Generate random crops from image batches.
    """
    while True:
        batch_x, batch_y = next(batches)
        num_channels = batch_x.shape[-1]  # Handle grayscale and RGB images
        batch_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, num_channels))
        for i in range(batch_x.shape[0]):
            batch_crops[i] = random_crop(batch_x[i], (crop_length, crop_length))
        yield (batch_crops, batch_y)

In [ ]:

def residual_block(x, number_of_filters, match_filter_size=False):
    """
    Residual block with convolutional layers.
    """
#     config = model_configuration()
    initializer = tf.keras.initializers.HeNormal() #config.get("initializer")

    # Create skip connection
    x_skip = x

    # Perform the original mapping
    if match_filter_size:
        x = Conv2D(number_of_filters, kernel_size=(3, 3), strides=(2,2), kernel_initializer=initializer, padding="same")(x_skip)
    else:
        x = Conv2D(number_of_filters, kernel_size=(3, 3), strides=(1,1), kernel_initializer=initializer, padding="same")(x_skip)
    x = BatchNormalization(axis=3)(x)
    x = Activation("relu")(x)
    x = Conv2D(number_of_filters, kernel_size=(3, 3), kernel_initializer=initializer, padding="same")(x)
    x = BatchNormalization(axis=3)(x)

    # Perform matching of filter numbers if necessary
    if match_filter_size and config.get("shortcut_type") == "identity":
        x_skip = Lambda(lambda x: tf.pad(x[:, ::2, ::2, :], tf.constant([[0, 0], [0, 0], [0, 0], [number_of_filters//4, number_of_filters//4]]), mode="CONSTANT"))(x_skip)
    elif match_filter_size and config.get("shortcut_type") == "projection":
        x_skip = Conv2D(number_of_filters, kernel_size=(1,1), kernel_initializer=initializer, strides=(2,2))(x_skip)

    # Add the skip connection to the regular mapping
    x = Add()([x, x_skip])

    # Nonlinearly activate the result
    x = Activation("relu")(x)

    return x


def ResidualBlocks(x):
    """
    Set up the residual blocks.
    """
#     config = model_configuration()

    # Set initial filter size
    filter_size = 16 #config.get("initial_num_feature_maps")

    for layer_group in range(3):
        for block in range(3): #config.get("stack_n")):
            if layer_group > 0 and block == 0:
                filter_size *= 2
                x = residual_block(x, filter_size, match_filter_size=True)
            else:
                x = residual_block(x, filter_size)

    return x



def model_base(shp):
    """
    Base structure of the model, with residual blocks attached.
    """
#     config = model_configuration()
    initializer = tf.keras.initializers.HeNormal() #config.get("initializer")

    inputs = Input(shape=shp)
    x = Conv2D(16, kernel_size=(3,3), strides=(1,1), kernel_initializer=initializer, padding="same")(inputs)
    x = BatchNormalization(axis=3)(x)
    x = Activation("relu")(x)

    x = ResidualBlocks(x)

    x = GlobalAveragePooling2D()(x)
#     x = Flattenn()(x)
    x = Dense(7, kernel_initializer=initializer)(x)

    model = Model(inputs=inputs, outputs=x)

    return model

def init_model():
    """
    Initialize the model with the specified configurations.
    """
    
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    optimizer_momentum = 0.9
    optimizer_additional_metrics = ["accuracy"]
    optimizer = SGD(learning_rate=lr_schedule, momentum=optimizer_momentum)
#     config = model_configuration()
#     width, height, dim = config.get("width"), config.get("height"), config.get("dim")

    model = model_base((width, height, dim))
    model.compile(optimizer=optimizer, loss=loss, metrics=optimizer_additional_metrics)
#     model.summary()
    return model

# train_batches, validation_batches, _ = preprocessed_dataset()
# train_batches = train_generator.flow(input_train, target_train, batch_size=32, subset="training")
# validation_batches
tensorboard = TensorBoard(
    log_dir=os.path.join(os.getcwd(), "logs"),
    histogram_freq=1,
    write_images=True
)

# Save a model checkpoint after every epoch
checkpoint = ModelCheckpoint(
    os.path.join(os.getcwd(), "model_checkpoint.keras"),
    save_freq="epoch"
)

# Add callbacks to list
callbacks = [
    tensorboard,
    checkpoint
]

steps_per_epoch = int(np.floor(train_size / batch_size))
val_steps_per_epoch = int(np.floor(val_size / batch_size))
model.summary()

In [ ]:
model = init_model()
history = model.fit(
    train_batches, 
    validation_data=validation_batches,
    epochs=100,
    callbacks=callbacks
)

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Add, Input, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD

def load_dataset():
    file_path = '/kaggle/input/fer2013/fer2013.csv'
    data = pd.read_csv(file_path)

    pixels = data['pixels'].tolist()
    width, height = 48, 48
    faces = []
    for pixel_sequence in pixels:
        face = np.asarray([int(pixel) for pixel in pixel_sequence.split(' ')]).reshape(width, height)
        face = np.stack((face,)*3, axis=-1)  # Convert to RGB
        faces.append(face)
    faces = np.array(faces)
    faces = faces.astype('float32') / 255.0

    emotions = pd.get_dummies(data['emotion']).values

    # Split the data into training and testing sets
    train_size = int(0.8 * len(faces))
    input_train, input_test = faces[:train_size], faces[train_size:]
    target_train, target_test = emotions[:train_size], emotions[train_size:]

    return (input_train, target_train), (input_test, target_test)

def residual_block(x, number_of_filters, match_filter_size=False):
    initializer = tf.keras.initializers.HeNormal()
    x_skip = x
    if match_filter_size:
        x = Conv2D(number_of_filters, kernel_size=(3, 3), strides=(2,2), kernel_initializer=initializer, padding="same")(x_skip)
    else:
        x = Conv2D(number_of_filters, kernel_size=(3, 3), strides=(1,1), kernel_initializer=initializer, padding="same")(x_skip)
    x = BatchNormalization(axis=3)(x)
    x = Activation("relu")(x)
    x = Conv2D(number_of_filters, kernel_size=(3, 3), kernel_initializer=initializer, padding="same")(x)
    x = BatchNormalization(axis=3)(x)

    if match_filter_size:
        x_skip = Conv2D(number_of_filters, kernel_size=(1,1), kernel_initializer=initializer, strides=(2,2))(x_skip)

    x = Add()([x, x_skip])
    x = Activation("relu")(x)
    return x

def ResidualBlocks(x):
    filter_size = 16
    for layer_group in range(3):
        for block in range(3):
            if layer_group > 0 and block == 0:
                filter_size *= 2
                x = residual_block(x, filter_size, match_filter_size=True)
            else:
                x = residual_block(x, filter_size)
    return x

def model_base(shp):
    initializer = tf.keras.initializers.HeNormal()
    inputs = Input(shape=shp)
    x = Conv2D(16, kernel_size=(3,3), strides=(1,1), kernel_initializer=initializer, padding="same")(inputs)
    x = BatchNormalization(axis=3)(x)
    x = Activation("relu")(x)

    x = ResidualBlocks(x)

    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)  # Add dropout for regularization
    x = Dense(7, kernel_initializer=initializer)(x)
    model = Model(inputs=inputs, outputs=x)
    return model

def init_model():
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    optimizer = SGD(learning_rate=0.01, momentum=0.9)
    model = model_base((48, 48, 3))
    model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])
    return model

(input_train, target_train), (input_test, target_test) = load_dataset()

train_generator = ImageDataGenerator(
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    shear_range=0.1,
    rescale=1./255
)

train_batches = train_generator.flow(input_train, target_train, batch_size=32, subset="training")
validation_batches = train_generator.flow(input_train, target_train, batch_size=32, subset="validation")

test_generator = ImageDataGenerator(rescale=1./255)
test_batches = test_generator.flow(input_test, target_test, batch_size=32)

tensorboard = TensorBoard(log_dir=os.path.join(os.getcwd(), "logs"), histogram_freq=1, write_images=True)
checkpoint = ModelCheckpoint(os.path.join(os.getcwd(), "model_checkpoint.keras"), save_freq="epoch")
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

callbacks = [tensorboard, checkpoint, early_stopping, reduce_lr]

model = init_model()
model.summary()
history = model.fit(
    train_batches,
    validation_data=validation_batches,
    epochs=100,
    callbacks=callbacks
)